In [75]:
test_string = "C12H6 O2 C1O2 H2O1"

numbers = []
for char in test_string:
    if char.isdigit():
        numbers.append(int(char))

print("The numbers list is:", numbers)

The numbers list is: [1, 2, 6, 2, 1, 2, 2, 1]


In [76]:
file = open("input.txt", "r")
content = file.read()
print(content)
file.close()

C H O
C2 H6 + O2 -> C O2 + H2 O


In [77]:
with open('input.txt') as f:
    first_line = f.readline()

print(first_line)

C H O



In [78]:
with open('input.txt') as f:
    Chemical_elements = f.readline().strip('\n')

print(Chemical_elements)

C H O


In [79]:
f = open("input.txt",'r')

#read the second line of the file
Chemical_equation = f.readlines()[1:2]
print(Chemical_equation)

#closes the file
f.close()

['C2 H6 + O2 -> C O2 + H2 O']


In [80]:
type(Chemical_equation)

list

Parsing the input

In [81]:
import numpy as np

def parse_input(filename):
    with open(filename, 'r') as file:
        elements = file.readline().strip().split()  # Read elements from the first line
        equation = file.readline().strip()  # Read chemical equation from the second line
    return elements, equation

import re
def extract_species(species_str):
    components = re.findall(r'([A-Za-z]+)(\d*)', species_str)
    species = {}
    for element, count_str in components:
        count = int(count_str) if count_str else 1
        species[element] = count
    return species

def create_element_matrix(species, elements):
    matrix = []
    for element in elements:
        count = species.get(element, 0)
        matrix.append([count])
    return np.array(matrix)

def main():
    filename = "input.txt"
    elements, equation = parse_input(filename)

    reactants_str, products_str = equation.split('->')
    reactants = reactants_str.split('+')
    products = products_str.split('+')

    reactant_matrices = []
    product_matrices = []

    for reactant in reactants:
        reactant_species = extract_species(reactant.strip())
        reactant_matrices.append(create_element_matrix(reactant_species, elements))

    max_reactant_rows = max(matrix.shape[0] for matrix in reactant_matrices)

    for product in products:
        product_species = extract_species(product.strip())
        product_matrix = create_element_matrix(product_species, elements)
        # Pad the product matrix with zeros to match the number of rows in the reactant matrices
        padded_product_matrix = np.pad(product_matrix, ((0, max_reactant_rows - product_matrix.shape[0]), (0, 0)), 'constant')
        # Multiply product matrix by -1 to make the elements negative
        padded_product_matrix *= -1
        product_matrices.append(padded_product_matrix)

    # Concatenate matrices horizontally
    combined_matrix = np.hstack(reactant_matrices + product_matrices)

    # Add a column of zeros
    zero_column = np.zeros((max_reactant_rows, 1))
    combined_matrix = np.hstack((combined_matrix, zero_column))

    return combined_matrix

   # print(combined_matrix.shape, combined_matrix.size, combined_matrix.dtype)


Augmented Matrix

In [82]:
if __name__ == "__main__":
    combined_matrix = main()
    print("Combined Matrix:")
    print(combined_matrix)

Combined Matrix:
[[ 2.  0. -1.  0.  0.]
 [ 6.  0.  0. -2.  0.]
 [ 0.  2. -2. -1.  0.]]


Row Reduction Algorithm

In [83]:
import numpy as np

# Function to apply row reduction (Gaussian elimination)
def row_reduction(matrix):
    m, n = matrix.shape
    i = 0
    for j in range(n):
        pivot_row = i
        # Find the pivot element
        while pivot_row < m and matrix[pivot_row, j] == 0:
            pivot_row += 1
        if pivot_row == m:
            continue  # If pivot is zero, move to the next column
        # Swap rows to bring pivot element to the current row
        matrix[[i, pivot_row]] = matrix[[pivot_row, i]]
        # Make the pivot element 1
        matrix[i] /= matrix[i, j]
        # Make all other elements in the current column zero
        for k in range(m):
            if k != i:
                matrix[k] -= matrix[k, j] * matrix[i]
        i += 1
        if i == m:
            break
    return matrix

# Example usage
if __name__ == "__main__":
    combined_matrix = main()
    print("Original Matrix:")
    print(combined_matrix)
    reduced_matrix = row_reduction(combined_matrix)
    print("\nRow-Reduced Matrix:")
    print(reduced_matrix)


Original Matrix:
[[ 2.  0. -1.  0.  0.]
 [ 6.  0.  0. -2.  0.]
 [ 0.  2. -2. -1.  0.]]

Row-Reduced Matrix:
[[ 1.          0.          0.         -0.33333333  0.        ]
 [ 0.          1.          0.         -1.16666667  0.        ]
 [ 0.          0.          1.         -0.66666667  0.        ]]


find basic and free variables and solve the linear equation

In [84]:
import numpy as np

def find_basic_and_free_variables(matrix):
    m, n = matrix.shape
    basic_indices = []
    free_indices = []
    for j in range(n - 1):  # Exclude the last column (the augmented column)
        column = matrix[:, j]
        if np.count_nonzero(column) == 1 and np.argmax(column) == j:
            basic_indices.append(j)
        else:
            free_indices.append(j)
    return basic_indices, free_indices

import numpy as np

def find_general_solution(matrix, basic_indices, free_indices):
    num_variables = matrix.shape[1] - 1  # Exclude the augmented column
    general_solution = {}

    # Loop through basic indices
    for i, basic_index in enumerate(basic_indices):
        variable_name = f'X{basic_index+1}'  # Basic variables are named as X1, X2, ...

        # Initialize the expression for the variable
        variable_expression = f"{matrix[i, -1]}"

        # Loop through free indices to add terms corresponding to free variables
        for j, free_index in enumerate(free_indices):
            if matrix[i, free_index] != 0:
                coefficient = matrix[i, free_index] / matrix[i, basic_index]
                variable_expression += f" + ({-coefficient} * X{free_index+1})"

        general_solution[variable_name] = variable_expression

    return general_solution

# Example usage
if __name__ == "__main__":
    combined_matrix = main()
    reduced_matrix = row_reduction(combined_matrix)

    print(basic_indices)
    print(free_indices)
    general_solution = find_general_solution(reduced_matrix, basic_indices, free_indices)
    print("\nGeneral Solution:")
    for variable, expression in general_solution.items():
        print(f"{variable} = {expression}")



[0, 1, 2]
[3]

General Solution:
X1 = 0.0 + (0.3333333333333333 * X4)
X2 = 0.0 + (1.1666666666666665 * X4)
X3 = 0.0 + (0.6666666666666666 * X4)
